# Lab 3: Securely Connect External Tools to Multi-Agent System with AgentCore Gateway

## Overview

In this lab, you will learn how to integrate tools available in your organization with your Multi-Agent Customer Support System using **Amazon Bedrock AgentCore Gateway**.

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro) is an open protocol that standardizes how applications provide tools and context to Large Language Models (LLMs).

With [Amazon Bedrock AgentCore Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html), developers can convert APIs, Lambda functions, and existing services into MCP-compatible tools and make them available to agents through Gateway endpoints with just a few lines of code.

**Workshop Journey:**
- **Lab 1 (Done)**: Multi-Agent Foundation - Built orchestrator with specialized agents
- **Lab 2 (Done)**: Multi-Agent Memory - Added persistent memory across agents
- **Lab 3 (Current)**: Multi-Agent Gateway - Secure tool sharing via AgentCore Gateway
- **Lab 4**: Multi-Agent Runtime - Deploy with observability and monitoring
- **Lab 5**: Multi-Agent Frontend - Build customer-facing application

### Why AgentCore Gateway & Tool Sharing Matter for Multi-Agent Systems

**Current State (Labs 1-2):** Each agent has its own copy of tools, leading to:
- Code duplication across different agents
- Inconsistent tool behavior and maintenance overhead
- No centralized security or access control
- Difficulty scaling to multiple use cases

**After this lab:** Centralized, reusable tools that can serve:
- Customer Support Agent (our current use case)
- Sales Agent (needs same product info and customer data)
- Inventory Agent (needs same product info and warranty checking)
- Returns Processing Agent (needs return policies and customer profiles)

### Adding Secure Authentication with AgentCore Identity

Additionally, AgentCore Gateway requires you to securely authenticate both inbound and outbound connections. [AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html) provides seamless agent identity and access management across AWS services and third-party applications such as Slack and Zoom while supporting any standard identity providers such as Okta, Entra, and Amazon Cognito. In this lab we will see how AgentCore Gateway integrates with AgentCore Identity to provide secure connections via inbound and outbound authentication.

For the inbound authentication, the AgentCore Gateway analyzes the OAuth token passed during invocation to decide allow or deny the access to a tool in the gateway. If a tool needs access to external resources, the AgentCore Gateway can use outbound authentication via API Key, IAM or OAuth Token to allow or deny the access to the external resource.

During the inbound authorization flow, an agent or the MCP client calls an MCP tool in the AgentCore Gateway adding an OAuth access token (generated from the user's IdP). AgentCore Gateway then validates the OAuth access token and performs inbound authorization.

If the tool running in AgentCore Gateway needs to access external resources, OAuth will retrieve credentials of downstream resources using the resource credential provider for the Gateway target. AgentCore Gateway pass the authorization credentials to the caller to get access to the downstream API.

## Multi-Agent Gateway Architecture for Lab 3

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

*Web search tool is now centralized in AgentCore Gateway with secure identity-based access control. Multiple agents and use cases can share the same tool securely. We will also reuse the `check_warranty()` tool built for other applications and add the `web_search()` tool for use within other applications. `get_product_info()`, `get_return_policy()`, and `get_technical_support` remain as local tools as they are specific to the customer support use case*

### Key Features
- **Seamlessly integrate AWS Lambda functions:** This example shows how to integrate your Agent with existing AWS Lambda functions to check the warranty of an item and to get the customer profile using Amazon Bedrock AgentCore Gateway.
- **Secure your Gateway endpoint with Inbound Auth**: Only an Agent providing a valid JWT token can connect to the endpoint to use the tools
- **Configure the Agent to use the MCP endpoint**: The Agent gets a valid JWT token and uses it to connect to the MCP endpoint provided by AgentCore Gateway

## Prerequisites

* Python 3.12+
* AWS credentials configured
* Amazon Nova Pro enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* Complete Lab 2 Add memory to the Multi-Agent System
* **Deploy prerequisite infrastructure** using `./scripts/prereq.sh`
* These resources are created for you by the prerequisite script:
    - AWS Lambda function 
    - AWS Lambda Execution IAM Role
    - AgentCore Gateway IAM Role
    - DynamoDB tables used by the AWS Lambda function. 
    - Cognito User Pool and User Pool Client

## Step 1: Install and import required libraries

In [1]:
# Install required packages
%pip install strands-agents "boto3>=1.39.15" strands-agents-tools bedrock_agentcore ddgs -q


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import os
import sys
import boto3
import json
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

from scripts.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret

sts_client = boto3.client('sts')

# Get AWS account details
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Step 2: Load API Specification for Lambda Tools

We need to load the tool schema which describes the tools implemented by our Lambda function for MCP compatibility.

In [3]:
try:
    api_spec_file = "./prerequisite/lambda/api_spec.json"

    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"❌ API specification file not found: {api_spec_file}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)
 
    print("✅ API specification loaded successfully")
    print(f"📋 Available tools: {[tool['name'] for tool in api_spec]}")
    
except Exception as e:
    print(f"❌ Error loading API specification: {str(e)}")
    sys.exit(1)

✅ API specification loaded successfully
📋 Available tools: ['check_warranty_status', 'web_search']


In [4]:
# Step 3: Fix Cognito Configuration and Get OAuth Token
import boto3

cognito = boto3.client('cognito-idp')
userpool_id = get_ssm_parameter("/app/reinvent/agentcore/userpool_id")
client_id = get_ssm_parameter("/app/reinvent/agentcore/machine_client_id")
auth_scope = get_ssm_parameter("/app/reinvent/agentcore/cognito_auth_scope")  # Add this line

print("🔧 Fixing Cognito User Pool Client configuration...")

try:
    # Get current client configuration
    response = cognito.describe_user_pool_client(
        UserPoolId=userpool_id,
        ClientId=client_id
    )
    
    client_config = response['UserPoolClient']
    
    print(client_config)
    # Update client to support client credentials flow
    cognito.update_user_pool_client(
        UserPoolId=userpool_id,
        ClientId=client_id,
        ClientName=client_config['ClientName'],
        AllowedOAuthFlows=['client_credentials'],
        AllowedOAuthScopes=[auth_scope],  # Use the SSM parameter instead of hardcoded value
        AllowedOAuthFlowsUserPoolClient=True,
        SupportedIdentityProviders=['COGNITO']
    )
    
    print("✅ User Pool Client updated for client credentials flow")

    
    # OAuth function
    def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,
        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()
    
    # Get authentication token
    gateway_access_token = get_token(
        client_id,
        get_cognito_client_secret(),
        get_ssm_parameter("/app/reinvent/agentcore/cognito_auth_scope"),
        get_ssm_parameter("/app/reinvent/agentcore/cognito_token_url")
    )
    
    print("🔐 Authentication token obtained successfully!")
    
except Exception as e:
    print(f"❌ Error: {e}")


🔧 Fixing Cognito User Pool Client configuration...
{'UserPoolId': 'us-east-1_gSqycONGx', 'ClientName': 'CustomerSupportMachineClient', 'ClientId': '5croohvqpjpa63g89k4n4qp0oi', 'ClientSecret': '1ckiatu4hoon7v8so1a7f87o6u814g3cefj37e3ddhst9tlpmdt', 'LastModifiedDate': datetime.datetime(2025, 10, 9, 15, 36, 47, 48000, tzinfo=tzlocal()), 'CreationDate': datetime.datetime(2025, 10, 9, 12, 48, 49, 503000, tzinfo=tzlocal()), 'RefreshTokenValidity': 1, 'AccessTokenValidity': 60, 'IdTokenValidity': 60, 'TokenValidityUnits': {'AccessToken': 'minutes', 'IdToken': 'minutes', 'RefreshToken': 'days'}, 'ExplicitAuthFlows': ['ALLOW_REFRESH_TOKEN_AUTH', 'ALLOW_USER_PASSWORD_AUTH', 'ALLOW_USER_SRP_AUTH'], 'AllowedOAuthFlowsUserPoolClient': False, 'EnableTokenRevocation': True, 'EnablePropagateAdditionalUserContextData': False, 'AuthSessionValidity': 3}
✅ User Pool Client updated for client credentials flow
🔐 Authentication token obtained successfully!


## Step 3: Create Your AgentCore Gateway

Now let's create the AgentCore Gateway to expose the Lambda function as MCP-compatible endpoint.

To validate the callers authorized to invoke our tools we need to configure the Inbound Auth.

Inbound Auth works using OAuth authorization, the standard for MCP servers. With OAuth the client application must authenticate with the OAuth authorizer before using the Gateway. Your client would receive an access token which is used at runtime.

You need to specify an OAuth discovery server and client IDs. The prerequisite script already provisioned the Cognito UserPool and UserPoolClient and stored the discovery URL and the Client ID in dedicated SSM parameters.

In [5]:
gateway_name = "reinvent-customersupport-gw"

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            get_ssm_parameter("/app/reinvent/agentcore/machine_client_id")
        ],
        "discoveryUrl": get_ssm_parameter("/app/reinvent/agentcore/cognito_discovery_url")
    }
}

try:
    # create new gateway
    print(f"Creating gateway in region {REGION} with name: {gateway_name}")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn= get_ssm_parameter("/app/reinvent/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/reinvent/agentcore/gateway_id", gateway_id)

    print(f"✅ Gateway created successfully with ID: {gateway_id}")

except Exception as e:
    # If gateway exists, collect existing gateway ID from SSM
    existing_gateway_id = get_ssm_parameter("/app/reinvent/agentcore/gateway_id")
    print(f"Found existing gateway with ID: {existing_gateway_id}")
    
    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
    gateway = {
        "id": existing_gateway_id,
        "name": gateway_response["name"],
        "gateway_url": gateway_response["gatewayUrl"],
        "gateway_arn": gateway_response["gatewayArn"],
    }
    gateway_id = gateway['id']

Creating gateway in region us-west-2 with name: reinvent-customersupport-gw
Found existing gateway with ID: reinvent-customersupport-gw-8euuj5msmj


## Step 4: Add the Lambda Function Target

Now we will use the previously defined function definitions to create a Lambda target within our Agent Gateway. This will define the tools that your gateway will host.

Gateway allows you to attach multiple targets to a Gateway and you can change the targets/tools attached to a gateway at any point. Each target can have its own credential provider, but Gateway becomes a single MCP URL enabling access to all of the relevant tools for an agent across myriad APIs.

In [6]:
import time

try:
    # Wait for gateway to become READY
    while True:
        response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
        status = response.get('status', response.get('gateway', {}).get('status'))
        print(f"Gateway status: {status}")
        
        if status in ['ACTIVE', 'READY']:
            break
        elif status == 'FAILED':
            raise Exception("Gateway creation failed")
            
        time.sleep(10)

    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter("/app/reinvent/agentcore/lambda_arn"),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }

    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ Gateway target created: {create_target_response['targetId']}")

except Exception as e:
    print(f"❌ Error creating gateway target: {str(e)}")


Gateway status: READY
❌ Error creating gateway target: An error occurred (ConflictException) when calling the CreateGatewayTarget operation: A target with name 'LambdaUsingSDK' already exists in this gateway


## Step 5: Set Up Secure MCP Client

Here we integrate our authentication token from Cognito into an MCPClient to create an MCP Server object for our Multi-Agent System.

In [7]:
# OAuth 2.0 Client Credentials flow for secure Gateway access
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    """Get OAuth token for Gateway authentication"""
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,
        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

# Get authentication token
gateway_access_token = get_token(
    get_ssm_parameter("/app/reinvent/agentcore/machine_client_id"),
    get_cognito_client_secret(),
    get_ssm_parameter("/app/reinvent/agentcore/cognito_auth_scope"),
    get_ssm_parameter("/app/reinvent/agentcore/cognito_token_url")
)

print(f"🔐 Authentication token obtained for Gateway access")
print(f"🌐 Gateway Endpoint - MCP URL: {gateway['gateway_url']}")

# Set up MCP client for secure Gateway communication
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

print("✅ MCP Client configured for secure Gateway access")

🔐 Authentication token obtained for Gateway access
🌐 Gateway Endpoint - MCP URL: https://reinvent-customersupport-gw-8euuj5msmj.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
✅ MCP Client configured for secure Gateway access


## Step 6: Import Multi-Agent System Components

Now we'll import our multi-agent system components and enhance them with Gateway tools.

In [8]:
# Import multi-agent components with compatibility layer
from lab_helpers.compatibility import (
    get_product_info, get_return_policy, get_technical_support, 
    SYSTEM_PROMPT, CustomerSupportMemoryHooks, create_or_get_memory_resource
)
import uuid
from bedrock_agentcore.memory import MemoryClient

print("✅ Multi-agent components imported successfully")

✅ Multi-agent components imported successfully


## Step 7: Create Enhanced Multi-Agent System with Gateway Tools

Now we'll create our enhanced multi-agent system that combines local tools with shared Gateway tools.

In [9]:
# Initialize memory and model
memory_client = MemoryClient(region_name=REGION)
memory_id = create_or_get_memory_resource()
SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_001"
memory_hooks = CustomerSupportMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Initialize the Bedrock model
model_id = "us.amazon.nova-pro-v1:0"
model = BedrockModel(
    model_id=model_id,
    temperature=0.3,  # Balanced between creativity and consistency
    region_name=REGION
)

# Start MCP client and get Gateway tools
try:
    mcp_client.start()
    gateway_tools = mcp_client.list_tools_sync()
    print(f"🔧 Retrieved {len(gateway_tools)} tools from Gateway")
except Exception as e:
    print(f"Error initializing MCP client: {str(e)}")
    gateway_tools = []

# Combine local and Gateway tools
tools = [
    get_product_info,
    get_return_policy,
    get_technical_support
] + gateway_tools

print(f"\n🛠️ Total tools available: {len(tools)}")
print("   Local tools: get_product_info, get_return_policy, get_technical_support")
print(f"   Gateway tools: {len(gateway_tools)} tools from Lambda functions")

# Create the enhanced multi-agent system
agent = Agent(
    model=model,
    tools=tools,
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("\n✅ Enhanced Multi-Agent System created successfully!")
print("🔗 Now using both local tools and secure Gateway tools")
print("🔐 Gateway tools are protected by OAuth authentication")
print("🌐 Tools can be shared across multiple agents and use cases")

🔧 Retrieved 2 tools from Gateway

🛠️ Total tools available: 5
   Local tools: get_product_info, get_return_policy, get_technical_support
   Gateway tools: 2 tools from Lambda functions

✅ Enhanced Multi-Agent System created successfully!
🔗 Now using both local tools and secure Gateway tools
🔐 Gateway tools are protected by OAuth authentication
🌐 Tools can be shared across multiple agents and use cases


## Step 8: Test the Enhanced Multi-Agent System

Let's test our enhanced system with queries that use both local and Gateway tools.

In [10]:
# Test Case 1: List available tools
print("\n" + "="*60)
print("Test Case 1: What tools do you have available?")
print("="*60)

response = agent("What tools do you have available to help me?")
print(response)

print("\n" + "-"*60)


Test Case 1: What tools do you have available?
<thinking> The customer is experiencing technical issues with their iPhone 14 and laptop, and is seeking troubleshooting support, detailed technical specifications, and information about return policies and warranties. I should provide them with information about the tools available to help with these issues. </thinking>

The tools available to help you are:

1. **get_product_info**: This tool provides detailed product information and specifications. You can use it to get information about the iPhone 14 or any other product you are interested in.

2. **get_return_policy**: This tool provides information about the return policy for products. You can use it to understand the return policy for smartphones or any other product type.

3. **get_technical_support**: This tool provides technical support information from the knowledge base. You can use it to get troubleshooting solutions for your iPhone 14 or laptop.

4. **LambdaUsingSDK___check_w

In [11]:
# Test Case 2: Product inquiry with warranty check (uses both local and Gateway tools)
print("\n" + "="*60)
print("Test Case 2: Product inquiry with warranty check")
print("="*60)

response = agent(
    "I'm interested in gaming headphones. Can you tell me about them and also check the warranty status for serial number GH123456789?"
)
print(response)

print("\n" + "-"*60)


Test Case 2: Product inquiry with warranty check
<thinking> The customer is interested in gaming headphones, particularly wireless ones, and wants to know about their technical specifications and warranty status. They have provided a serial number (GH123456789) for the warranty check. I should use the available tools to provide the requested information. </thinking>


Tool #1: get_product_info

Tool #2: LambdaUsingSDK___check_warranty_status
<thinking> The customer requested information about wireless gaming headphones and the warranty status for a specific serial number. The tool results indicate that the product information for a wireless gaming headset is available, but the warranty status for the provided serial number (GH123456789) was not found in the database. I should inform the customer about these results and suggest next steps. </thinking>

Here is the information you requested:

**Product Information:**
- **Gaming Headset X1:** Wireless, 40ms latency, $199

**Warranty Stat

In [12]:
# Test Case 3: Technical issue with web search (uses Gateway tools)
print("\n" + "="*60)
print("Test Case 3: Technical issue with web search")
print("="*60)

response = agent(
    "My iPhone 14 is overheating. Can you help me troubleshoot this and also search for the latest solutions online?"
)
print(response)

print("\n" + "-"*60)


Test Case 3: Technical issue with web search
<thinking> The customer is experiencing overheating issues with their iPhone 14 and is seeking troubleshooting support. I should provide them with step-by-step troubleshooting solutions and search for the latest solutions online. </thinking>


Tool #3: get_technical_support

Tool #4: LambdaUsingSDK___web_search
<thinking> The customer is experiencing overheating issues with their iPhone 14. I have retrieved troubleshooting solutions from the knowledge base and the latest solutions from an online search. I should provide the customer with these solutions and suggest further steps if the issue persists. </thinking>

Here are the troubleshooting steps and latest solutions for your iPhone 14 overheating issue:

**Troubleshooting Steps:**
1. **Check Ventilation:** Ensure that the iPhone's vents are not blocked.
2. **Clean Fans:** If applicable, clean any fans or cooling components.
3. **Monitor CPU Usage:** Check for any apps that may be causing

## Summary

In this lab, you successfully:

1. **Created AgentCore Gateway** - Set up a secure MCP endpoint for tool sharing
2. **Implemented Lambda Tools** - Built warranty checking and web search functions
3. **Configured OAuth Authentication** - Secured Gateway access with Cognito
4. **Enhanced Multi-Agent System** - Combined local and Gateway tools
5. **Tested Integration** - Verified secure tool access across agents

### Key Takeaways

- **Tool Centralization**: Gateway enables sharing tools across multiple agents and use cases
- **Security**: OAuth and IAM provide robust access control for agent tools
- **Scalability**: MCP standard ensures compatibility with any agent framework
- **Maintainability**: Centralized tools reduce duplication and improve consistency

### Next Steps

In **Lab 4**, you'll deploy this enhanced multi-agent system to production using AgentCore Runtime with observability and monitoring capabilities.

The Gateway tools you created here will be available to all future agents in your organization, providing a solid foundation for scaling your multi-agent architecture.